<a href="https://colab.research.google.com/github/khushboosharma94/Big-data_Capstone_Project/blob/main/Banking_Data_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

--2023-12-21 08:58:54--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz’

spark-3.0.0-bin-had 100%[===================>] 210.07M  1.52MB/s    in 1m 42s  

2023-12-21 09:00:36 (2.07 MB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hadoop2.7/k

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("DemoApp").master("local").config('spark.jars','sqlite-jdbc-3.44.1.0.jar').getOrCreate()
db_properties = {"driver":"org.sqlite.JDBC"}
connection_url = "jdbc:sqlite:/content/customers.db"
customers_df= spark.read.jdbc(connection_url,'customers',properties=db_properties)
customers_df.printSchema()
customers_df.show()
-----To update column name----
updated_column_names =['customer_id','first_name','last_name','date_of_birth']
cus_df = customers_df.toDF(*updated_column_names)
cus_df.show()



root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- date_of_birth: string (nullable = true)

+---+----------+---------+-------------+
| id|first_name|last_name|date_of_birth|
+---+----------+---------+-------------+
|  1|      John|      Doe|   1980-05-15|
|  2|      Jane|    Smith|   1992-08-21|
|  3|     Alice|  Johnson|   1975-02-10|
|  4|     Sarah|    Jones|   1988-12-03|
|  5|     David|    Brown|   1995-04-18|
|  6|      Emma|   Miller|   1982-07-25|
+---+----------+---------+-------------+

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          1|      John|      Doe|   1980-05-15|
|          2|      Jane|    Smith|   1992-08-21|
|          3|     Alice|  Johnson|   1975-02-10|
|          4|     Sarah|    Jones|   1988-12-03|
|          5|     David|    Brown|   1995-04-18|
|          6|      Emma|   M

In [12]:
import json
import csv

with open('accounts.json') as json_file1:
    data1 = json.load(json_file1)

accounts_data = data1['accounts']
data_file1 = open('accounts.csv', 'w')

csv_writer1 = csv.writer(data_file1)

count1 = 0

for a in accounts_data:
    if count1 == 0:
        header = a.keys()
        csv_writer1.writerow(header)
        count1 += 1
    csv_writer1.writerow(a.values())

data_file1.close()
acc_df = spark.read.csv(path='accounts.csv',header=True)
acc_df.show()
acc_df.printSchema()

+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
|         1|          1|          2|     Savings|   5000|
|         2|          1|          3|    Checking|   1000|
|         3|          2|          4|     Savings|   8000|
|         4|          3|          1|    Checking|   3000|
|         5|          2|          2|    Checking|   2500|
|         6|          3|          3|     Savings|   6000|
|         7|          4|          4|    Checking|  12000|
|         8|          5|          2|     Savings|   3000|
+----------+-----------+-----------+------------+-------+

root
 |-- account_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- balance: string (nullable = true)



In [6]:
dfr= spark.read
file_path = 'loans.csv'
loan= dfr.csv('loans.csv', header=True, inferSchema=True)
loan.show()
loan.printSchema()

+-------+-----------+-----------+-------------+----------+----------+------+
|loan_id|customer_id|loan_amount|interest_rate|start_date|  end_date|status|
+-------+-----------+-----------+-------------+----------+----------+------+
|      1|          1|      10000|         0.05|2023-01-01|2023-12-31|Active|
|      2|          1|      15000|         0.04|2023-02-15|2023-12-31|Active|
|      3|          2|       8000|         0.03|2023-03-20|2023-11-30|Active|
|      4|          3|      20000|         0.05|2023-01-10|2023-10-31|Active|
+-------+-----------+-----------+-------------+----------+----------+------+

root
 |-- loan_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- status: string (nullable = true)



In [7]:
dfr= spark.read
file_path = 'payment_history.csv'
payment_history= dfr.csv('payment_history.csv', header=True, inferSchema=True)
payment_history.show()
payment_history.printSchema()



+----------+-------+------------+-----------+
|payment_id|loan_id|payment_date|amount_paid|
+----------+-------+------------+-----------+
|      null|      1|  2023-03-01|       2000|
|      null|      1|  2023-04-01|       1500|
|      null|      2|  2023-04-10|       1000|
|      null|      3|  2023-02-20|       3000|
|      null|      3|  2023-03-15|       2500|
+----------+-------+------------+-----------+

root
 |-- payment_id: string (nullable = true)
 |-- loan_id: integer (nullable = true)
 |-- payment_date: string (nullable = true)
 |-- amount_paid: integer (nullable = true)



In [8]:
dfr= spark.read
file_path = 'transactions.csv'
transaction_data= dfr.csv('transactions.csv', header=True, inferSchema=True)
transaction_data.show()
transaction_data.printSchema()


+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|
|             4|         3|      Withdrawal|  1000|2023-04-05 10:10:00|
|             5|         4|         Deposit|  1500|2023-05-20 09:00:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------

In [ ]:
from pyspark.sql.functions import col, explode
accounts_df = spark.read.option("multiLine", True).json("/content/branches.json")
# Explode the nested array of accounts
exploded_accounts_df = accounts_df.select(explode("branches").alias("branch"))
accounts_df_1 = exploded_accounts_df.select(
    col("branch.branch_id").alias("branch_id"),
    col("branch.branch_name").alias("branch_name"),
    col("branch.location").alias("location"),
    ).show()
exploded_accounts_df.printSchema()

+---------+---------------+-------------+
|branch_id|    branch_name|     location|
+---------+---------------+-------------+
|        1|    Main Branch|Downtown City|
|        2|Suburban Branch|  Suburbville|
|        3|Regional Branch|Regional City|
+---------+---------------+-------------+

root
 |-- branch: struct (nullable = true)
 |    |-- branch_id: long (nullable = true)
 |    |-- branch_name: string (nullable = true)
 |    |-- location: string (nullable = true)



In [ ]:
dfr= spark.read
file_path = 'employees.csv'
employee_data= dfr.csv('employees.csv', header=True, inferSchema=True)
employee_data.show(30)
employee_data.printSchema()

+-----------+---------+----------+---------+--------+
|employee_id|branch_id|first_name|last_name|position|
+-----------+---------+----------+---------+--------+
|          1|        2|      Mike|  Johnson| Manager|
|          2|        3|     Emily| Williams|  Teller|
|          3|        2|    Robert|    Davis|  Teller|
|          4|        3|    Olivia|   Wilson|  Teller|
|          5|        2|    Daniel|  Johnson| Analyst|
|          6|        3|    Sophia|    Clark| Manager|
|          7|        2|      Mike|  Johnson| Manager|
|          8|        3|     Emily| Williams|  Teller|
|          9|        2|    Robert|    Davis|  Teller|
|         10|        3|    Olivia|   Wilson|  Teller|
|         11|        2|    Daniel|  Johnson| Analyst|
|         12|        3|    Sophia|    Clark| Manager|
+-----------+---------+----------+---------+--------+

root
 |-- employee_id: integer (nullable = true)
 |-- branch_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |

**Summary of the Reports Required**

**Problem statement 1**

**Write a spark dataframe to show the balance amount for an account_id = 1:**

In [41]:
from pyspark.sql.functions import *
balance_amount = acc_df.select('balance').filter(col('account_id')==1)
balance_amount.show()

+-------+
|balance|
+-------+
|   5000|
+-------+



## **Problem Statement - 2**

List Transactions for an account_id = 1

In [42]:
dataframe1 = transaction_data.filter(col('account_id')==1)
dataframe1.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------------+



## **Problem Statement - 3**

List Accounts with a zero balance:

In [43]:
from pyspark.sql.functions import *
zero_balance = acc_df.filter(col('balance')==0)
zero_balance.show()

+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
+----------+-----------+-----------+------------+-------+



#**Problem statement 4**

**Find the Oldest Customer:**

In [10]:
from pyspark.sql.window import Window

customers_df.show()

Oldest_cust= customers_df.withColumn('date_of_birth_date', to_date(from_unixtime(unix_timestamp(col('date_of_birth'), 'yyyy-MM-dd')))) \
.withColumn('min', min(col('date_of_birth_date')).over(Window.orderBy('date_of_birth_date'))).filter(col('date_of_birth_date')==col('min')).drop('min')

Oldest_cust.show()

+---+----------+---------+-------------+
| id|first_name|last_name|date_of_birth|
+---+----------+---------+-------------+
|  1|      John|      Doe|   1980-05-15|
|  2|      Jane|    Smith|   1992-08-21|
|  3|     Alice|  Johnson|   1975-02-10|
|  4|     Sarah|    Jones|   1988-12-03|
|  5|     David|    Brown|   1995-04-18|
|  6|      Emma|   Miller|   1982-07-25|
+---+----------+---------+-------------+

+---+----------+---------+-------------+------------------+
| id|first_name|last_name|date_of_birth|date_of_birth_date|
+---+----------+---------+-------------+------------------+
|  3|     Alice|  Johnson|   1975-02-10|        1975-02-10|
+---+----------+---------+-------------+------------------+



**Calculate the Total Interest Earned Across All Accounts:**

**#ACCOUNTS REPORTS**

**1. List All Accounts with Customer Information:**

In [22]:
cus_df.show()
acc_df.show()
df7= cus_df.join(acc_df,'customer_id','inner').select('customer_id','first_name','last_name','account_id','account_type','balance').sort(col('account_id'))
df7.show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          1|      John|      Doe|   1980-05-15|
|          2|      Jane|    Smith|   1992-08-21|
|          3|     Alice|  Johnson|   1975-02-10|
|          4|     Sarah|    Jones|   1988-12-03|
|          5|     David|    Brown|   1995-04-18|
|          6|      Emma|   Miller|   1982-07-25|
+-----------+----------+---------+-------------+

+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
|         1|          1|          2|     Savings|   5000|
|         2|          1|          3|    Checking|   1000|
|         3|          2|          4|     Savings|   8000|
|         4|          3|          1|    Checking|   3000|
|         5|          2|          2|    Checking|   2500|
|         6|          3|          3|     Savi

**2. Calculate Total Balance for Each Customer:**

In [25]:
df7.show()
df8 = df7.groupBy('customer_id','first_name','last_name').agg(sum('balance').alias('total')).sort(col('customer_id'))
df8.show()

+-----------+----------+---------+----------+------------+-------+
|customer_id|first_name|last_name|account_id|account_type|balance|
+-----------+----------+---------+----------+------------+-------+
|          1|      John|      Doe|         1|     Savings|   5000|
|          1|      John|      Doe|         2|    Checking|   1000|
|          2|      Jane|    Smith|         3|     Savings|   8000|
|          3|     Alice|  Johnson|         4|    Checking|   3000|
|          2|      Jane|    Smith|         5|    Checking|   2500|
|          3|     Alice|  Johnson|         6|     Savings|   6000|
|          4|     Sarah|    Jones|         7|    Checking|  12000|
|          5|     David|    Brown|         8|     Savings|   3000|
+-----------+----------+---------+----------+------------+-------+

+-----------+----------+---------+-------+
|customer_id|first_name|last_name|  total|
+-----------+----------+---------+-------+
|          1|      John|      Doe| 6000.0|
|          2|      Jane

**3. Find Customers with Multiple Accounts:**

In [31]:
acc_df.show()
cus_Multi_acc=acc_df.groupBy('customer_id').count().sort('customer_id')
cus_Multi_acc.show()
updated_column_names = ['customer_id','num_accounts']
Multi_accounts = cus_Multi_acc.toDF(*updated_column_names)
Multi_accounts.show()


+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
|         1|          1|          2|     Savings|   5000|
|         2|          1|          3|    Checking|   1000|
|         3|          2|          4|     Savings|   8000|
|         4|          3|          1|    Checking|   3000|
|         5|          2|          2|    Checking|   2500|
|         6|          3|          3|     Savings|   6000|
|         7|          4|          4|    Checking|  12000|
|         8|          5|          2|     Savings|   3000|
+----------+-----------+-----------+------------+-------+

+-----------+-----+
|customer_id|count|
+-----------+-----+
|          1|    2|
|          2|    2|
|          3|    2|
|          4|    1|
|          5|    1|
+-----------+-----+

+-----------+------------+
|customer_id|num_accounts|
+-----------+------------+
|          1|           2|
|          2| 

# **Customer Transactions Reports**

**1. List Transactions with Account and Customer Information:**

In [33]:
list= transaction_data.join(acc_df,'account_id').join(cus_df,'customer_id')\
.select('transaction_id','transaction_type','amount','transaction_date','customer_id','first_name','last_name','account_id','account_type')\
.sort(col('transaction_id'),ascending=True)
list.show()

+--------------+----------------+------+-------------------+-----------+----------+---------+----------+------------+
|transaction_id|transaction_type|amount|   transaction_date|customer_id|first_name|last_name|account_id|account_type|
+--------------+----------------+------+-------------------+-----------+----------+---------+----------+------------+
|             1|         Deposit|  1000|2023-01-15 08:30:00|          1|      John|      Doe|         1|     Savings|
|             2|      Withdrawal|   500|2023-02-02 12:45:00|          1|      John|      Doe|         1|     Savings|
|             3|         Deposit|  2000|2023-03-10 15:20:00|          1|      John|      Doe|         2|    Checking|
|             4|      Withdrawal|  1000|2023-04-05 10:10:00|          2|      Jane|    Smith|         3|     Savings|
|             5|         Deposit|  1500|2023-05-20 09:00:00|          3|     Alice|  Johnson|         4|    Checking|
|             6|         Deposit|  2000|2023-06-12 11:30

**2 Calculate Average Transaction Amount:**

In [34]:
avg_transactions= list.agg(avg(col('amount')).alias('avg_transaction_amt'))

avg_transactions.show()

+-------------------+
|avg_transaction_amt|
+-------------------+
|             1450.0|
+-------------------+



**3 Identify High-Value Customers with Total Balance:**

In [36]:
High_value = cus_df.join(acc_df,'customer_id')\
.groupBy('customer_id','first_name','last_name').agg(sum('balance').alias('total_balance')).sort(col('customer_id').asc())
High_value.show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|total_balance|
+-----------+----------+---------+-------------+
|          1|      John|      Doe|       6000.0|
|          2|      Jane|    Smith|      10500.0|
|          3|     Alice|  Johnson|       9000.0|
|          4|     Sarah|    Jones|      12000.0|
|          5|     David|    Brown|       3000.0|
+-----------+----------+---------+-------------+



**5 Calculate the Total Number of Transactions for Each Account Type:**

In [24]:
df14 = acc_df.groupBy('account_type').count()\
.withColumnRenamed('count','num_transaction')
df14.show()

+------------+---------------+
|account_type|num_transaction|
+------------+---------------+
|     Savings|              4|
|    Checking|              4|
+------------+---------------+



**7. Find Customers with No Accounts:**

In [37]:
no_accounts = cus_df.join(acc_df,'customer_id','left_anti')\
.select('customer_id','first_name','last_name')
no_accounts.show()

+-----------+----------+---------+
|customer_id|first_name|last_name|
+-----------+----------+---------+
|          6|      Emma|   Miller|
+-----------+----------+---------+



**9. Calculate the Total Withdrawals for Each customer:**

In [13]:
transaction_data.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|
|             4|         3|      Withdrawal|  1000|2023-04-05 10:10:00|
|             5|         4|         Deposit|  1500|2023-05-20 09:00:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------

In [40]:
transaction_data.show()
Total_withdraw = transaction_data.filter(col('transaction_type')=='Withdrawal')\
.groupBy('account_id','transaction_type')\
.agg(sum('amount').alias('total_withdrawals'))

Totalwithdraw=Total_withdraw.join(acc_df,'account_id','left').join(cus_df,'customer_id','left')\
.select('customer_id','first_name','last_name','total_withdrawals')\
.sort(col('customer_id').asc())

Totalwithdraw.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|
|             4|         3|      Withdrawal|  1000|2023-04-05 10:10:00|
|             5|         4|         Deposit|  1500|2023-05-20 09:00:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------

 **10** **Find Duplicate Transactions:**

In [ ]:
drop_duplicate = transaction_data.exceptAll(transaction_data.dropDuplicates())
drop_duplicate.show()

+--------------+----------+----------------+------+----------------+
|transaction_id|account_id|transaction_type|amount|transaction_date|
+--------------+----------+----------------+------+----------------+
+--------------+----------+----------------+------+----------------+

